# Finite Element Analysis
Solve equations to calculate mechanical equilibrium in FEA. The theoretical background of the exercise is covered in Section 2.2 of the script to this lecture.

In [1]:
import numpy as np

## Task 1
Fill in the missing code segments marked by "..." for the B-matrix and the integration of the stiffness matrix, refers to script Section 2.6, Eq. (2).

In [2]:
def Bmatrix(x,L):
    # Evaluate B matrix at given point x within 1-d element with quadratic shape function
    B = np.zeros((6,3))
    h1 = 1./L
    h2 = 4./(L*L)
    # Missing code segments here:
    B[0,0] = ...
    B[0,1] = ...
    B[0,2] = ...
    return B
    
def calc_Kel(Lel):
    # Calculate element stiffness matrix by Gaussian integration
    cpos = np.sqrt(1./3.)   # position of Gauss points
    wght = 1
    Vel  = A*Lel
    x1 = 0.5*Lel*(1. - cpos)  # positions of first Gauss point
    x2 = 0.5*Lel*(1. + cpos)  # positions of second Gauss point
    B1 = Bmatrix(x1,Lel)
    B2 = Bmatrix(x2,Lel)
    K1 = np.transpose(B1) @ CV @ B1
    # Missing code segments here
    K2 = ...
    Kel = ...
    return Kel

## Task 2

Set the elastic constants in the following code according to Assignment 1 and execute the cell. This will result in the finite element solutions for nodal displacements and forces for mechanical equilibrium under the applied load. Compare this finite element code to the one for the system of springs considering how the system stiffness matrix is setup and how displacement boundary conditions are enforced.

In [4]:
# Setup Finite Element Model for 1-d elements with quadratic shape function
N = 1        # number of elements
M = 2*N + 1  # DOF
Lel = 1.     # list of lengths of elements (must have dimension N)
A = 1.       # cross-sectional area of elements

# Define material parameters
C11 = ...
C12 = ...
C44 = ...
CV = np.array([[C11, C12, C12, 0.,  0.,  0.], \
               [C12, C11, C12, 0.,  0.,  0.], \
               [C12, C12, C11, 0.,  0.,  0.], \
               [0.,  0.,  0.,  C44, 0.,  0.], \
               [0.,  0.,  0.,  0.,  C44, 0.], \
               [0.,  0.,  0.,  0.,  0.,  C44]])

# Initialize system arrays
F = np.zeros(M)
u = np.zeros(M)
K = np.zeros((M,M))

# calculate element stiffness matrix
for i in range(N):
    # calculate element stiffness matrix for element #i
    Kel = calc_Kel(Lel)
    # insert element stiffness matrix into system stiffness matrix
    K[2*i:2*i+3,2*i:2*i+3] += Kel  

print('System stiffness matrix for ',N,'elements:\n', K)

# Apply boundary conditions
u[0] = 0.
u[M-1] = np.sum(Lel)*0.01  # apply 1% of total strain

for i in range(1,M-1):
    # Reduce rank of system of equations for given displacement BC
    F[i] -= K[i,0]*u[0]  
    F[i] -= K[i,M-1]*u[M-1]
    
# Solve reduced system of equations
u[1:M-1] = np.linalg.solve(K[1:M-1,1:M-1], F[1:M-1])
print('After solving reduced system of equations:')
print('u: ',u, '\nF: ',F)
F = K@u # calculate reaction force
print('After calculation of reaction forces:')
print('u: ',u, '\nF: ',F)

System stiffness matrix for  1 elements:
 [[ 628133.33333333 -717866.66666667   89733.33333333]
 [-717866.66666667 1435733.33333333 -717866.66666667]
 [  89733.33333333 -717866.66666667  628133.33333333]]
After solving reduced system of equations:
u:  [0.    0.005 0.01 ] 
F:  [   0.         7178.66666667    0.        ]
After calculation of reaction forces:
u:  [0.    0.005 0.01 ] 
F:  [-2.69200000e+03 -1.13056231e-13  2.69200000e+03]


## Answer:

Fill in your answer here.

## Task 3

Fill in the missing code segments marked by "..." for the definition of strain and stress. This task refers to the script, Section 2.4, Eqs. (28) and (29). Compare the stress obtained for a uniaxial strain of 1% to the value in your solution of Assignment 1.

In [5]:
'Calculate stress and strain'
xi = 0.5
for i in range(N):
    L = Lel
    x = xi*L
    B = Bmatrix(x,L)
    eps = ... @ u[2*i:2*i+3]
    sig = ... @ eps
    print('Strain tensor in element #', i, ': eps = (', eps,')')
    print('Stress tensor in element #', i, ': sig = (', sig,') MPa')


Strain tensor in element # 0 : eps = ( [0.01 0.   0.   0.   0.   0.  ] )
Stress tensor in element # 0 : sig = ( [2692. 1154. 1154.    0.    0.    0.] ) MPa


## Answer: 

Fill in your answer here.

## Task 4

Use the `pyLabFEA` package to setup a 1-d model for a material with the elastic constants of Assignment 1 and apply a uniaxial strain of 1%. The online documentation for the `pyLabFEA` package is avaiable under https://ahartmaier.github.io/pyLabFEA/ and also on the Moodle course. The package needs to be downloaded and installed from https://github.com/AHartmaier/pyLabFEA.git or with the ZIP acrchive in the Moodle course. Please to go through the tutorial contained in the Jupyter notebook ``pyLabFEA_Introduction.ipynb`` before completing this task. Compare the results for the stress tensor obtained with the pyLabFEA model to the results of the simple model above. What is the unit of the nodal forces?

In [ ]:
import pylabfea as FE

# Define model parameters
E  = ...     # Young's modulus in MPa
nu = 0.3     # Poisson ratio
length = 1.  # total length of model im mm
Nel = 1      # number of elements

# Define material
mat = FE.Material()
mat.elasticity(E=E, nu=nu)
print('Elastic constants: C11=%6.2fGPa, C12=%6.2fGPa, C44=%6.2fGPa'
      %(mat.C11*1e-3, mat.C12*1e-3, mat.C44*1e-3))

# Define finite element model
fem = FE.Model()
fem.geom([length])
fem.assign([mat])
fem.bcleft(0.)
# apply displacement on rhs node corresponding to 1% total strain
fem.bcright(0.01*fem.lenx, 'disp') 
# mesh 1-d model with one element with quadratic shape function
fem.mesh(NX=Nel, SF=2)
# solve equations for mechanical equilibrium under given BC
fem.solve()
print('\nNodal displacements u = ', fem.u.round(decimals=4), 'mm')
print('Nodal forces f = ',fem.f.round(decimals=2),'unit???')
print('Element strain eps_1 = ',
      [element.eps[0].round(decimals=2) for element in fem.element])
print('Element stress sig_1 = ',
      [element.sig[0].round(decimals=2) for element in fem.element],'MPa')
print('Voigt strain tensor for element #0 = ',
      fem.element[0].eps.round(decimals=2))
print('Voigt stress tensor for element #0 = ',
      fem.element[0].sig.round(decimals=2),'MPa')

print('\nGraphical output of model with magnification factor 1')
fem.plot('strain1', mag=1)


## Answer:

Fill in your answer here.

## Task 5

Create a 2-D plane strain finite element model (standard setting) and apply the strain boundary conditions (BC) of Assignment 1, Task 3b (only the first two loading states can be applied, because simple shear boundary conditions are not yet implemented in `pyLabFEA`). Which kind of BC (`'disp'` or `'force'`) do you need to apply to the model in order to get uniaxial or biaxial strains? Compare the results to those of the assignment and the 1-d model above.

In [ ]:
# Define 2-d plane strain finite element model (standard setting)
f2d = FE.Model(dim=2)
f2d.geom([length])
f2d.assign([mat])

# Load case 1
ubc = ...       # boundary node displacement corresponding to 1% of strain
bctype = ...    # type of BC 'force' or 'disp'
f2d.bcleft(0.)   # displacement BC at left boundary
f2d.bcbot(0.)    # displacement BC at bottom boundary
f2d.bcright(ubc, bctype) # apply displacement on rhs node corresponding to 1% total strain
f2d.bctop(0., bctype)
f2d.mesh(NX=2, NY=2)   # mesh 2-d model with 4 quadrilateral elements with linear shape function
f2d.solve() 
print('\n2-d Model: uniaxial strain')
print('Global strain: ',f2d.glob['eps'][0].round(decimals=4))
print('Element strain: ', f2d.element[0].eps.round(decimals=4))
print('Global stress: ',f2d.glob['sig'][0].round(decimals=2),'MPa')
print('Element stress: ', f2d.element[0].sig.round(decimals=2),'MPa')

f2d.plot('strain1', mag=100)
f2d.plot('stress1', mag=100)

# Load case 2
f2d.u = None  # reset solution
f2d.bcleft(0.)
f2d.bcbot(0.)
f2d.bcright(ubc/np.sqrt(2.), bctype) # apply displacement on rhs node corresponding to 1% total strain
f2d.bctop(ubc/np.sqrt(2.),bctype)
f2d.mesh(NX=2, NY=2)   # mesh 2-d model with 4 quadrilateral elements with linear shape function
f2d.solve() 
print('\n2-d Model: biaxial strain')
print('Global strain: ',f2d.glob['eps'][0].round(decimals=4))
print('Element strain: ', f2d.element[0].eps.round(decimals=4))
print('Global stress: ',f2d.glob['sig'][0].round(decimals=2),'MPa')
print('Element stress: ', f2d.element[0].sig.round(decimals=2),'MPa')

f2d.plot('strain1', mag=100)
f2d.plot('stress1', mag=100)


## Answer:

Fill in your answer here.

## Task 6

Create a 2-D finite element model with plane stress conditions and apply the stress boundary conditions (BC) of Assignment 1, Task 3a (only the first two loading states can be applied, because shear boundary conditions are not yet implemented in `pyLabFEA`). Which kind of BC (`'disp'` or `'force'`) do you need to apply to the model get uniaxial or biaxial stresses? Why is a plane stress condition used here? Compare the results to those of the assignment.

In [ ]:
# Define 2-d plane stress finite element model
f2d = FE.Model(dim=2, planestress=True)
f2d.geom([length])
f2d.assign([mat])

# load case 1
sig0 = 100. # stress to be applied
A = f2d.leny*f2d.thick   # cross-sectional area of model (can be changed in geom)
F0 = ...     # force required to get desired stress
bctype = ... # type of BC 'force' or 'disp'
f2d.bcleft(0.)   # displacement BC at left boundary
f2d.bcbot(0.)    # displacement BC at bottom boundary
f2d.bcright(F0, bctype) # apply BC 100 MPa
f2d.bctop(0., bctype)
f2d.mesh(NX=2, NY=2)   # mesh 2-d model with 4 quadrilateral elements with linear shape function
f2d.solve() 
print('\n2-d Model: uniaxial stress')
print('Global strain: ',f2d.glob['eps'][0].round(decimals=4))
print('Element strain: ', f2d.element[0].eps.round(decimals=4))
print('Global stress: ',f2d.glob['sig'][0].round(decimals=2),'MPa')
print('Element stress: ', f2d.element[0].sig.round(decimals=2),'MPa')

f2d.plot('strain1', mag=100)
f2d.plot('stress1', mag=100)

# Load case 2
f2d.u = None  # reset solution
f2d.bcleft(0.)
f2d.bcbot(0.)
f2d.bcright(F0, bctype) # apply force to nodes on right
f2d.bctop(F0, bctype)   # apply force to nodes on top
f2d.mesh(NX=2, NY=2)   # mesh 2-d model with 4 quadrilateral elements with linear shape function
f2d.solve() 
print('\n2-d Model: biaxial stress')
print('Global strain: ',f2d.glob['eps'][0].round(decimals=7))
print('Element strain: ', f2d.element[0].eps.round(decimals=7))
print('Global stress: ',f2d.glob['sig'][0].round(decimals=2),'MPa')
print('Element stress: ', f2d.element[0].sig.round(decimals=2),'MPa')

f2d.plot('strain1', mag=100)
f2d.plot('stress1', mag=100)


## Answer:

Fill in your answer here.